## 使用openMind Library Trainer微调PyTorch预训练模型 ​
openMind Library提供了一个对Transformers Trainer类的封装，您可以像使用Transformers Trainer类一样来使用openMind Library Trainer类，您无需手动编写自己的训练循环步骤就可以开始训练模型。Trainer API支持各种训练选项和功能，如日志记录、梯度累积和上传模型等。

### 准备数据集 

在微调预训练模型之前，请下载数据集并准备进行训练。 当前数据集的操作依然使用Hugging Face所提供的datasets库。

从加载Yelp Reviews数据集开始：

In [ ]:
from datasets import load_dataset

dataset = load_dataset("yelp_review_full")
print(dataset["train"][100])

'''
输出：
{
   'label': 0,
   'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that '
          'takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to '
          'force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited '
          'over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two '
          'people who ordered after me be handed their food, I asked where mine was. The manager started yelling '
          'at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither '
          'cashier was anywhere near those controls, and the manager was the one serving food to customers and '
          'clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had '
          'everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor '
          'service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. I\'ve worked at more than '
          'one location. I expect bad days, bad moods, and the occasional mistake. But I have yet to have a decent '
          'experience at this store. It will remain a place I avoid unless someone in my party needs to avoid '
          'illness from low blood sugar. Perhaps I should go back to the racially biased service of Steak n Shake '
          'instead!'}
'''


需要使用tokenizer来处理文本，包括填充和截断操作以处理可变的序列长度

In [ ]:
from openmind import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openmind/bert_base_cased_pt")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)


### 模型初始化 

首先加载模型并指定期望的标签数量，Yelp Review数据集有五个标签。

In [ ]:
from openmind import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("openmind/bert_base_cased_pt", num_labels=5)


### 参数初始化

创建一个TrainingArguments类，其中包含可以调整的所有超参数以及不同的训练选项。可以查看Trainer模块接口文档，尝试并找到最优的设置。

In [ ]:
from openmind import TrainingArguments

# 指定保存训练检查点的路径
training_args = TrainingArguments(output_dir="test_trainer")


### 配置评估参数 

Trainer在训练过程中不会自动评估模型性能，需要向Trainer传递一个函数来计算和展示指标。 当前评估操作依然使用Hugging Face所提供的Evaluate库。Evaluate库提供了一个简单的accuracy函数，可以使用evaluate.load函数加载。

In [ ]:
import evaluate

metric = evaluate.load("accuracy")


在metric上调用evaluate.compute来计算模型预测的准确性：将预测结果传递给compute方法之前，需要将预测结果转换为预测标签，最终计算出准确率accuracy。

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


要在微调过程中监视评估指标，需指定训练参数evaluation_strategy设置为"epoch"，以便在每个epoch结束时展示当前的评估指标。

In [ ]:
from openmind import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")


### 创建训练器并启动训练 

创建一个Trainer对象，将上述配置过的模型、训练超参数、训练/测试数据集以及评估函数传递给该对象。

In [ ]:
from openmind import Trainer

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = small_train_dataset,
    eval_dataset = small_eval_dataset,
    compute_metrics = compute_metrics,
)


调用openmind.Trainer.train以微调模型。

In [ ]:
trainer.train()
